#从对应的txt文件中读取word向量表和训练数据集
#embedding:words对应的150维向量(float)
#train_words:训练并列短语(string)
#train_types:训练并列短语每个词的词性(string)
#train_labels:训练并列短语对应的标记(int)

In [156]:
import pandas as pd
import numpy as np
import math
import theano
from sklearn import datasets
import pylab
import theano.tensor as T




In [160]:

'''
#load embedding
embedding = pd.read_csv('embedding.csv')
embedding.index = embedding['Unnamed: 0'].tolist()
del embedding['Unnamed: 0']


#load training context
train_context_f = open('coordinate_data_150/train/context.txt')
train_context_input = train_context_f.readlines()
n = 0
train_words = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
train_types = [(["unknown"]*6) for i in range(int(len(train_context_input)/3))]
words_index = 0
types_index = 0
while n < len(train_context_input):
    t = train_context_input[n].split()
    if n%3==0:
        for i in range(0,6):
            train_words[words_index][i] = t[i]
        words_index = words_index + 1
    elif n%3==1:
        for i in range(0,6):                                
            train_types[types_index][i] = t[i]
        types_index = types_index + 1
    n = n + 1
    
#load training labels
train_label_f = open('coordinate_data_150/train/labels.txt')
train_label_input = train_label_f.readlines()
n = 0
train_labels = [([-1]*3) for i in range(len(train_label_input))]
while n < len(train_label_input):
    t = train_label_input[n].split()
    for i in range(0,3):
        train_labels[n][i] = int(t[i])
    n = n + 1
'''
print('load data')

load data


In [161]:
#embedding.ix[train_types[0][0]]


In [164]:
'''
#计算余弦距离的函数
def cosine_dis(vector1,vector2):
    
    #@vector1 and @vector2 are two input vectors
    #@Return the cosine distance between two vectors
    
   
    mm = 0
    for i in range(0,len(vector1)):
        mm = mm + vector1[i]*vector2[i]
    dd1 = 0
    dd2 = 0
    for i in range(0,len(vector1)):
        dd1 = dd1 + vector1[i]*vector1[i]
        dd2 = dd2 + vector2[i]*vector2[i]
    dd1 = math.sqrt(dd1)
    dd2 = math.sqrt(dd2)
    result = mm/(dd1*dd2)
    return result
'''
print('cos')

cos


In [165]:
#TODO: 根据词性来训练
#余弦距离的定义
#距离度量标准？？？

    
#第一步：单纯以context中对应位置的词性作为训练标准
#第二步：在词性的基础上加入词汇
#第三步：选取更多的度量方法


In [166]:


train_num = 150000 #len(train_words); #训练量




In [167]:


#train_type_dis = [([float]*15) for i in range(train_num)]
#train_words_dis = [([float]*15) for i in range(train_num)]


In [168]:
#type和words的150维向量
#train_type_vector = pd.DataFrame(train_types)
#train_words_vector = pd.DataFrame(train_words)



In [170]:
'''
for i in range(0,(train_num)):
    for j in range(0,6):
        if train_types[i][j] in embedding.index:
            train_type_vector.ix[i][j]=embedding.ix[train_types[i][j]]
    if i%1000==0:
        print(i)
                                                
#train_type_vector.to_csv('train_type_vector.csv')
                
#TODO 用0取代缺失值

for i in range(0,(train_num)):
    for j in range(0,6):
        if train_words[i][j] in embedding.index:
            train_words_vector.ix[i][j]=embedding.ix[train_words[i][j]]
        else:
            train_words_vector.ix[i][j] = np.zeros(150)
    if i%1000==0:
        print(i)

#train_words_vector.to_csv('train_words_vector.csv')


for i in range(0,train_num):
    
    train_words_dis[i][0] = cosine_dis(train_words_vector.ix[i][0],train_words_vector.ix[i][1])
    train_words_dis[i][1] = cosine_dis(train_words_vector.ix[i][0],train_words_vector.ix[i][2])
    train_words_dis[i][2] = cosine_dis(train_words_vector.ix[i][0],train_words_vector.ix[i][3])
    train_words_dis[i][3] = cosine_dis(train_words_vector.ix[i][0],train_words_vector.ix[i][4])
    train_words_dis[i][4] = cosine_dis(train_words_vector.ix[i][0],train_words_vector.ix[i][5])
    train_words_dis[i][5] = cosine_dis(train_words_vector.ix[i][1],train_words_vector.ix[i][2])
    train_words_dis[i][6] = cosine_dis(train_words_vector.ix[i][1],train_words_vector.ix[i][3])
    train_words_dis[i][7] = cosine_dis(train_words_vector.ix[i][1],train_words_vector.ix[i][4])
    train_words_dis[i][8] = cosine_dis(train_words_vector.ix[i][1],train_words_vector.ix[i][5])
    train_words_dis[i][9] = cosine_dis(train_words_vector.ix[i][2],train_words_vector.ix[i][3])
    train_words_dis[i][10] = cosine_dis(train_words_vector.ix[i][2],train_words_vector.ix[i][4])
    train_words_dis[i][11] = cosine_dis(train_words_vector.ix[i][2],train_words_vector.ix[i][5])
    train_words_dis[i][12] = cosine_dis(train_words_vector.ix[i][3],train_words_vector.ix[i][4])
    train_words_dis[i][13] = cosine_dis(train_words_vector.ix[i][3],train_words_vector.ix[i][5])
    train_words_dis[i][14] = cosine_dis(train_words_vector.ix[i][4],train_words_vector.ix[i][5])
    
    
    if(i%10==0):
        print(i)
        
for i in range(0,train_num):
    
    train_type_dis[i][0] = cosine_dis(train_type_vector.ix[i][0],train_type_vector.ix[i][1])
    train_type_dis[i][1] = cosine_dis(train_type_vector.ix[i][0],train_type_vector.ix[i][2])
    train_type_dis[i][2] = cosine_dis(train_type_vector.ix[i][0],train_type_vector.ix[i][3])
    train_type_dis[i][3] = cosine_dis(train_type_vector.ix[i][0],train_type_vector.ix[i][4])
    train_type_dis[i][4] = cosine_dis(train_type_vector.ix[i][0],train_type_vector.ix[i][5])
    train_type_dis[i][5] = cosine_dis(train_type_vector.ix[i][1],train_type_vector.ix[i][2])
    train_type_dis[i][6] = cosine_dis(train_type_vector.ix[i][1],train_type_vector.ix[i][3])
    train_type_dis[i][7] = cosine_dis(train_type_vector.ix[i][1],train_type_vector.ix[i][4])
    train_type_dis[i][8] = cosine_dis(train_type_vector.ix[i][1],train_type_vector.ix[i][5])
    train_type_dis[i][9] = cosine_dis(train_type_vector.ix[i][2],train_type_vector.ix[i][3])
    train_type_dis[i][10] = cosine_dis(train_type_vector.ix[i][2],train_type_vector.ix[i][4])
    train_type_dis[i][11] = cosine_dis(train_type_vector.ix[i][2],train_type_vector.ix[i][5])
    train_type_dis[i][12] = cosine_dis(train_type_vector.ix[i][3],train_type_vector.ix[i][4])
    train_type_dis[i][13] = cosine_dis(train_type_vector.ix[i][3],train_type_vector.ix[i][5])
    train_type_dis[i][14] = cosine_dis(train_type_vector.ix[i][4],train_type_vector.ix[i][5])
    
    
    if(i%10==0):
        print(i)

train_type_dis_temp = pd.DataFrame(train_type_dis)
train_words_dis_temp = pd.DataFrame(train_words_dis)
#train_type_dis_temp.to_csv('train_type_dis.csv')
#train_words_dis_temp.to_csv('train_words_dis.csv')
#aaaa = pd.read_csv('train_words_dis.csv')
'''

print(train_num)

150000


In [171]:
#train_Y = [([int]) for i in range(train_num)]
#for i in range(train_num):
#    train_Y[i] = train_labels[i][2]


In [18]:
word_df1 = pd.read_csv('train_words_dis1.csv')
word_df2 = pd.read_csv('train_words_dis2.csv')
word_df2.index = word_df2.index+50000
word_df3 = pd.read_csv('train_words_dis3.csv')
word_df3.index = word_df3.index+100000
train_word_df = pd.concat([word_df1,word_df2,word_df3])
del train_word_df['Unnamed: 0']

In [19]:
train_word_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.078186,0.078186,0.093237,0.093237,0.187492,1.000000,0.082442,0.082442,0.566941,0.082442,0.082442,0.566941,1.000000,0.406256,0.406256
1,0.078186,0.078186,0.093237,0.140478,0.053903,1.000000,0.082442,0.043238,-0.547686,0.082442,0.043238,-0.547686,0.133389,-0.062813,0.161728
2,0.179263,0.043238,0.133389,0.133389,0.120529,0.150082,0.081872,0.081872,0.200082,0.082442,0.082442,0.566941,1.000000,0.406256,0.406256
3,0.652996,0.150082,0.081872,0.081872,0.200082,0.078186,0.093237,0.093237,0.187492,0.082442,0.082442,0.566941,1.000000,0.406256,0.406256
4,0.179263,0.043238,0.133389,0.120529,1.000000,0.150082,0.081872,0.200082,0.179263,0.082442,0.566941,0.043238,0.406256,0.133389,0.120529
5,0.652996,0.150082,0.081872,0.200082,0.179263,0.078186,0.093237,0.187492,0.140478,0.082442,0.566941,0.043238,0.406256,0.133389,0.120529
6,0.179263,0.043238,0.133389,1.000000,0.161728,0.150082,0.081872,0.179263,-0.049452,0.082442,0.043238,-0.547686,0.133389,-0.062813,0.161728
7,0.179263,0.043238,0.133389,1.000000,0.161728,0.150082,0.081872,0.179263,-0.049452,0.082442,0.043238,-0.547686,0.133389,-0.062813,0.161728
8,0.652996,0.150082,0.081872,0.179263,-0.049452,0.078186,0.093237,0.140478,0.053903,0.082442,0.043238,-0.547686,0.133389,-0.062813,0.161728
9,0.078186,0.078186,0.093237,0.187492,0.140478,1.000000,0.082442,0.566941,0.043238,0.082442,0.566941,0.043238,0.406256,0.133389,0.120529


In [22]:
type_df1 = pd.read_csv('train_type_dis1.csv')
type_df2 = pd.read_csv('train_type_dis2.csv')
type_df2.index = type_df2.index+50000
type_df3 = pd.read_csv('train_type_dis3.csv')
type_df3.index = type_df3.index+100000
train_type_df = pd.concat([type_df1,type_df2,type_df3])
del train_type_df['Unnamed: 0']

In [23]:
train_type_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.186247,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.186247,0.186247,0.186247,-0.111000,0.373484,1.000000,1.000000,0.490511,0.368714,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
2,-0.111000,0.490511,0.490511,0.490511,0.490511,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,0.186247,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,-0.111000,0.490511,0.490511,0.490511,1.000000,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511
5,1.000000,0.186247,0.186247,0.186247,-0.111000,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511
6,-0.111000,0.490511,0.490511,1.000000,0.219010,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
7,-0.111000,0.490511,0.490511,1.000000,0.219010,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
8,1.000000,0.186247,0.186247,-0.111000,0.373484,0.186247,0.186247,-0.111000,0.373484,1.000000,0.490511,0.368714,0.490511,0.368714,0.219010
9,0.186247,0.186247,0.186247,0.186247,-0.111000,1.000000,1.000000,1.000000,0.490511,1.000000,1.000000,0.490511,1.000000,0.490511,0.490511


In [11]:
train_Y = [([int]) for i in range(train_num)]
for i in range(train_num):
    train_Y[i] = train_labels[i][2]



In [175]:
#神经网络模型


#设置参数  
nn_input_dim=8 #输入神经元个数  
nn_output_dim=2 #输出神经元个数  
nn_hdim=50  #隐层节点数
epsilon=0.2 #learning rate  
reg_lambda=0.01 #正则化长度  

#参数初始化，设置为shared加速计算
w1=theano.shared(np.random.randn(nn_input_dim,nn_hdim),name="W1")  
b1=theano.shared(np.zeros(nn_hdim),name="b1")  
w2=theano.shared(np.random.randn(nn_hdim,nn_output_dim),name="W2")  
b2=theano.shared(np.zeros(nn_output_dim),name="b2")  

#前馈算法  
X=T.matrix('X')  #double类型的矩阵 
y=T.lvector('y') #int64类型的向量  
z1=X.dot(w1)+b1   #1 输入和w1的加权和
a1=T.tanh(z1)     #2 激活函数
z2=a1.dot(w2)+b2  #3 隐层输出和w2的加权和  
y_hat=T.nnet.softmax(z2) #4 激活函数  
loss_reg=1./train_num * reg_lambda/2 * (T.sum(T.square(w1))+T.sum(T.square(w2))) #5 正则化项    
loss=T.nnet.categorical_crossentropy(y_hat,y).mean() + loss_reg  #6 损失函数    
prediction=T.argmax(y_hat,axis=1) #7 预测结果  

forword_prop=theano.function([X],y_hat)  
calculate_loss=theano.function([X,y],loss)  
predict=theano.function([X],prediction)  

#求导  
dw2=T.grad(loss,w2)  
db2=T.grad(loss,b2)  
dw1=T.grad(loss,w1)  
db1=T.grad(loss,b1)  
#更新值  
gradient_step=theano.function(  
    [X,y],  
    updates=(  
        (w2,w2-epsilon*dw2),  
        (b2,b2-epsilon*db2),  
        (w1,w1-epsilon*dw1),  
        (b1,b1-epsilon*db1)  
  
    )  
)  

def build_model(num_passes=5000,print_loss=False):  
    w1.set_value(np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim))  
    b1.set_value(np.zeros(nn_hdim))  
    w2.set_value(np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim))  
    b2.set_value(np.zeros(nn_output_dim))  
    for i in range(0,num_passes):  
        gradient_step(train_X,train_Y)   #TODO change here
        if print_loss and i%1==0:  
            print("Loss after iteration %i: %f"%(i,calculate_loss(train_X,train_Y)))  

In [90]:
# 1 2 3 4 5 6
#指标1：2 and 4
#指标2：3 and 5
#指标3：1-2 and 1-4
#指标4：3-6 and 5-6

#word和type总共8个指标

#train_word_df.ix[:,0]-train_word_df.ix[:,0]
train_X = np.ndarray(shape=(train_num,8))
for i in range(train_num):
    train_X[i][0] = train_word_df.ix[i][6]
    train_X[i][1] = train_word_df.ix[i][6]
    train_X[i][2] = train_word_df.ix[i][0] - train_word_df.ix[i][2]
    train_X[i][3] = train_word_df.ix[i][11] - train_word_df.ix[i][14]
    train_X[i][4] = train_type_df.ix[i][6]
    train_X[i][5] = train_type_df.ix[i][6]
    train_X[i][6] = train_type_df.ix[i][0] - train_type_df.ix[i][2]
    train_X[i][7] = train_type_df.ix[i][11] - train_type_df.ix[i][14]
    if i %1000 ==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000


In [93]:
#replace nan with 0
for i in range(train_num):
    for j in range(8):
        if np.isnan(train_X[i][j]):
            train_X[i][j] = 0
            



In [176]:
build_model(print_loss=True)

Loss after iteration 0: 0.663115
Loss after iteration 1: 0.657555
Loss after iteration 2: 0.653248
Loss after iteration 3: 0.649838
Loss after iteration 4: 0.647100
Loss after iteration 5: 0.644874
Loss after iteration 6: 0.643042
Loss after iteration 7: 0.641517
Loss after iteration 8: 0.640235
Loss after iteration 9: 0.639145
Loss after iteration 10: 0.638211
Loss after iteration 11: 0.637403
Loss after iteration 12: 0.636699
Loss after iteration 13: 0.636080
Loss after iteration 14: 0.635532
Loss after iteration 15: 0.635043
Loss after iteration 16: 0.634604
Loss after iteration 17: 0.634207
Loss after iteration 18: 0.633846
Loss after iteration 19: 0.633516
Loss after iteration 20: 0.633213
Loss after iteration 21: 0.632932
Loss after iteration 22: 0.632671
Loss after iteration 23: 0.632427
Loss after iteration 24: 0.632198
Loss after iteration 25: 0.631982
Loss after iteration 26: 0.631778
Loss after iteration 27: 0.631583
Loss after iteration 28: 0.631398
Loss after iteration 29:

In [94]:
train_X


array([[ 0.08244176,  0.08244176, -0.01505103, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.08244176,  0.08244176, -0.01505103, ...,  1.        ,
         0.        ,  0.14970347],
       [ 0.08187206,  0.08187206,  0.04587453, ...,  0.18624691,
        -0.60151098,  0.        ],
       ..., 
       [-0.10657906, -0.10657906,  0.37886877, ..., -0.91218657,
        -1.10760129,  0.3649651 ],
       [-0.14468427, -0.14468427, -0.32197362, ..., -0.91218657,
        -1.91218657,  0.        ],
       [ 0.03928163,  0.03928163, -0.14240259, ..., -0.82519865,
        -0.91342413,  0.07467359]])

In [ ]:
'''
#load test

#load test context
test_context_f = open('coordinate_data_150/test/context.txt')
test_context_input = test_context_f.readlines()
n = 0
test_words = [(["unknown"]*6) for i in range(int(len(test_context_input)/3))]
test_types = [(["unknown"]*6) for i in range(int(len(test_context_input)/3))]
words_index = 0
types_index = 0
while n < len(test_context_input):
    t = test_context_input[n].split()
    if n%3==0:
        for i in range(0,6):
            test_words[words_index][i] = t[i]
        words_index = words_index + 1
    elif n%3==1:
        for i in range(0,6):                                
            test_types[types_index][i] = t[i]
        types_index = types_index + 1
    n = n + 1
    
#load test labels
test_label_f = open('coordinate_data_150/test/labels.txt')
test_label_input = test_label_f.readlines()
n = 0
test_labels = [([-1]*3) for i in range(len(test_label_input))]
while n < len(test_label_input):
    t = test_label_input[n].split()
    for i in range(0,3):
        test_labels[n][i] = int(t[i])
    n = n + 1

test_num = 10000  #测试量
test_type_dis = [([float]*15) for i in range(test_num)]
test_words_dis = [([float]*15) for i in range(test_num)]

#type和words的150维向量
test_type_vector = pd.DataFrame(test_types)
test_words_vector = pd.DataFrame(test_words)

for i in range(0,(test_num)):
    for j in range(0,6):
        if test_types[i][j] in embedding.index:
            test_type_vector.ix[i][j]=embedding.ix[test_types[i][j]]
    if i%1000==0:
        print(i)
        

for i in range(0,(test_num)):
    for j in range(0,6):
        if test_words[i][j] in embedding.index:
            test_words_vector.ix[i][j]=embedding.ix[test_words[i][j]]
        else:
            test_words_vector.ix[i][j] = np.zeros(150)
    if i%1000==0:
        print(i)
        
'''

In [ ]:
'''
for i in range(0,test_num):
    
    test_words_dis[i][0] = cosine_dis(test_words_vector.ix[i][0],test_words_vector.ix[i][1])
    test_words_dis[i][1] = cosine_dis(test_words_vector.ix[i][0],test_words_vector.ix[i][2])
    test_words_dis[i][2] = cosine_dis(test_words_vector.ix[i][0],test_words_vector.ix[i][3])
    test_words_dis[i][3] = cosine_dis(test_words_vector.ix[i][0],test_words_vector.ix[i][4])
    test_words_dis[i][4] = cosine_dis(test_words_vector.ix[i][0],test_words_vector.ix[i][5])
    test_words_dis[i][5] = cosine_dis(test_words_vector.ix[i][1],test_words_vector.ix[i][2])
    test_words_dis[i][6] = cosine_dis(test_words_vector.ix[i][1],test_words_vector.ix[i][3])
    test_words_dis[i][7] = cosine_dis(test_words_vector.ix[i][1],test_words_vector.ix[i][4])
    test_words_dis[i][8] = cosine_dis(test_words_vector.ix[i][1],test_words_vector.ix[i][5])
    test_words_dis[i][9] = cosine_dis(test_words_vector.ix[i][2],test_words_vector.ix[i][3])
    test_words_dis[i][10] = cosine_dis(test_words_vector.ix[i][2],test_words_vector.ix[i][4])
    test_words_dis[i][11] = cosine_dis(test_words_vector.ix[i][2],test_words_vector.ix[i][5])
    test_words_dis[i][12] = cosine_dis(test_words_vector.ix[i][3],test_words_vector.ix[i][4])
    test_words_dis[i][13] = cosine_dis(test_words_vector.ix[i][3],test_words_vector.ix[i][5])
    test_words_dis[i][14] = cosine_dis(test_words_vector.ix[i][4],test_words_vector.ix[i][5])
    
    
    if(i%10==0):
        print(i)
        
test_words_dis_temp = pd.DataFrame(test_words_dis)
test_words_dis_temp.to_csv('test_words_dis1.csv')


for i in range(0,test_num):
    
    test_type_dis[i][0] = cosine_dis(test_type_vector.ix[i][0],test_type_vector.ix[i][1])
    test_type_dis[i][1] = cosine_dis(test_type_vector.ix[i][0],test_type_vector.ix[i][2])
    test_type_dis[i][2] = cosine_dis(test_type_vector.ix[i][0],test_type_vector.ix[i][3])
    test_type_dis[i][3] = cosine_dis(test_type_vector.ix[i][0],test_type_vector.ix[i][4])
    test_type_dis[i][4] = cosine_dis(test_type_vector.ix[i][0],test_type_vector.ix[i][5])
    test_type_dis[i][5] = cosine_dis(test_type_vector.ix[i][1],test_type_vector.ix[i][2])
    test_type_dis[i][6] = cosine_dis(test_type_vector.ix[i][1],test_type_vector.ix[i][3])
    test_type_dis[i][7] = cosine_dis(test_type_vector.ix[i][1],test_type_vector.ix[i][4])
    test_type_dis[i][8] = cosine_dis(test_type_vector.ix[i][1],test_type_vector.ix[i][5])
    test_type_dis[i][9] = cosine_dis(test_type_vector.ix[i][2],test_type_vector.ix[i][3])
    test_type_dis[i][10] = cosine_dis(test_type_vector.ix[i][2],test_type_vector.ix[i][4])
    test_type_dis[i][11] = cosine_dis(test_type_vector.ix[i][2],test_type_vector.ix[i][5])
    test_type_dis[i][12] = cosine_dis(test_type_vector.ix[i][3],test_type_vector.ix[i][4])
    test_type_dis[i][13] = cosine_dis(test_type_vector.ix[i][3],test_type_vector.ix[i][5])
    test_type_dis[i][14] = cosine_dis(test_type_vector.ix[i][4],test_type_vector.ix[i][5])
    
    
    if(i%10==0):
        print(i)


test_type_dis_temp = pd.DataFrame(test_type_dis)
test_type_dis_temp.to_csv('test_type_dis1.csv')

'''

In [118]:
test_word_df = pd.read_csv('test_words_dis1.csv')
del test_word_df['Unnamed: 0']
test_type_df = pd.read_csv('test_type_dis1.csv')
del test_type_df['Unnamed: 0']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000,-0.440037,1.000000,1.000000,1.000000,-0.440037,1.0,1.000000,-0.440037,1.000000,-0.440037,-0.440037
5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
6,1.000000,1.000000,1.000000,1.000000,-0.144094,1.000000,1.000000,1.000000,-0.144094,1.0,1.000000,-0.144094,1.000000,-0.144094,-0.144094
7,-0.753955,-0.753955,-0.753955,-0.753955,-0.753955,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
8,0.717141,-0.899872,-0.899872,-0.899872,-0.899872,-0.753955,-0.753955,-0.753955,-0.753955,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
9,1.000000,1.000000,1.000000,-0.440037,0.641622,1.000000,1.000000,-0.440037,0.641622,1.0,-0.440037,0.641622,-0.440037,0.641622,-0.376751


In [119]:
test_X = np.ndarray(shape=(test_num,8))
for i in range(test_num):
    test_X[i][0] = test_word_df.ix[i][6]
    test_X[i][1] = test_word_df.ix[i][6]
    test_X[i][2] = test_word_df.ix[i][0] - test_word_df.ix[i][2]
    test_X[i][3] = test_word_df.ix[i][11] - test_word_df.ix[i][14]
    test_X[i][4] = test_type_df.ix[i][6]
    test_X[i][5] = test_type_df.ix[i][6]
    test_X[i][6] = test_type_df.ix[i][0] - test_type_df.ix[i][2]
    test_X[i][7] = test_type_df.ix[i][11] - test_type_df.ix[i][14]
    if i %1000 ==0:
        print(i)
        
#replace nan with 0
for i in range(test_num):
    for j in range(8):
        if np.isnan(test_X[i][j]):
            test_X[i][j] = 0
            
test_Y = [([int]) for i in range(test_num)]
for i in range(test_num):
    test_Y[i] = test_labels[i][2]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [177]:
error_num = 0

temp_y = predict(test_X)

for i in range(0,test_num):
    if(temp_y[i] != test_Y[i]):
        error_num = error_num+1
        
        

In [178]:
error_num

3378